<a href="https://colab.research.google.com/github/2813/ODS-homework/blob/main/06_ODS_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Градиентный бустинг**

## **Подготовка для работы в Google Colab или Kaggle**

#### Код для подключения Google Drive в Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Код для получения пути к файлам в Kaggle

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#### Код для установки библиотек

In [ ]:
%pip install numpy==1.26.4 pandas==2.1.4 scikit-learn==1.7.0 matplotlib==3.8.0 seaborn==0.13.2 catboost==1.2.8 lightgbm==4.6.0 xgboost==3.0.2 optuna==4.4.0

## **Важная информация**

**Для правильного воспроизведения результатов** решения задач:

* Рекомендуется придерживаться имеющего в заданиях кода в исходной последовательности. Для этого при решении задач **восстановите недостающие фрагменты кода, которые отмечены символом** `...` (Ellipsis).

* Если класс, функция или метод предусматривает параметр random_state, всегда указывайте **random_state=RANDOM_STATE**.

* Для всех параметров (кроме random_state) класса, функции или метода **используйте значения по умолчанию, если иное не указано в задании**.

**Если скорость обучения слишком низкая**, рекомендуется следующее:

* В модели или/и GridSearchCV поменяйте значение параметра n_jobs, который отвечает за параллелизм вычислений.

* Воспользуйтесь вычислительными ресурсами Google Colab или Kaggle.

***Использовать GPU не рекомендуется, поскольку результаты обучения некоторых моделей могут отличаться на CPU и GPU.***

После выполнения каждого задания **ответьте на вопросы в тесте.**

**ВНИМАНИЕ:** **После каждого нового запуска ноутбука** перед тем, как приступить к выполнению заданий, проверьте настройку виртуального окружения, выполнив код в ячейке ниже.

In [ ]:
# Код для проверки настройки виртуального окружения

import sys
from importlib.metadata import version

required = {
    'python': '3.11.x',
    'numpy': '1.26.4',
    'pandas': '2.1.4',
    'scikit-learn': '1.7.0',
    'matplotlib': '3.8.0',
    'seaborn': '0.13.2',
    'catboost': '1.2.8',
    'lightgbm': '4.6.0',
    'xgboost': '3.0.2',
    'optuna': '4.4.0'
}

print(f'{"Компонент":<15} | {"Требуется":<12} | {"Установлено":<12} | {"Соответствие"}')
print('-' * 62)

environment_ok = True
for lib, req_ver in required.items():
    try:
        if lib == 'python':
            inst_ver = sys.version.split()[0]
            status = '✓' if sys.version_info.major == 3 and sys.version_info.minor == 11 else f'x (требуется {req_ver})'
        else:
            inst_ver = version(lib)
            if inst_ver == req_ver:
                status = '✓'
            else:
                environment_ok = False
                status = f'x (требуется {req_ver})'
    except:
        environment_ok = False
        inst_ver = '-'
        status = 'x (не установлена)'
    print(f'{lib:<15} | {req_ver:<12} | {inst_ver:<12} | {status:<12}')

print('\nРезультат проверки: ',
      '✓\nВсе версии соответствуют требованиям'
      if environment_ok else
      'x\nВНИМАНИЕ: Версии некоторых компонентов не соответствуют требованиям!\n'
      'Для решения проблемы обратитесь к инструкции по настройке виртуального окружения')

## **Импорт библиотек и вспомогательные функции**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, classification_report, roc_auc_score, accuracy_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import optuna
from optuna.importance import get_param_importances
from optuna.samplers import TPESampler

In [ ]:
RANDOM_STATE = 42

## **Практическая часть**

### **Градиентный бустинг**

Градиентный бустинг (Gradient Boosting) — это метод машинного обучения, основанный на ансамбле слабых моделей, которые последовательно улучшают предсказания друг друга. В отличие от случайного леса, где деревья строятся независимо, в градиентном бустинге над решающими деревьями каждое новое дерево обучается на ошибках предыдущих.

**Алгоритм обучения:**

1. Инициализируется начальное предсказание $f_{0}(X)$, где $X$ — матрица признаков. Например:

    * Метка наиболее популярного класса (для задач классификации).

    * Среднее значение или ноль (для задач регрессии).

2. Для каждой итерации обучения $t=1,...,T$:

    1. Вычисляются остатки (градиент ошибки): $r_{t}=y-f_{t-1}(X)$, где $y$ — истинные значения целевой переменной.

    2. Обучается дерево решений $h_{t}(X)$​ для предсказания остатков $r_{t}$.

    3. Модель обновляется: $f_{t}(X)=f_{t-1}(X)+\gamma h_{t}(X)$, где $\gamma$ — темп (шаг) обучения.

Подробнее можно изучить по **ссылкам:**

* [Градиентный бустинг | education.yandex.ru](https://education.yandex.ru/handbook/ml/article/gradientnyj-busting)

* [All You Need to Know about Gradient Boosting Algorithm − Part 1. Regression | medium.com](https://medium.com/data-science/all-you-need-to-know-about-gradient-boosting-algorithm-part-1-regression-2520a34a502)

* [All You Need to Know about Gradient Boosting Algorithm − Part 2. Classification | towardsdatascience.com](https://medium.com/p/d3ed8f56541e)

### ***Задание 1***

Сгенерируйте простой набор данных для регрессии с одной объясняющей переменной (см. код):

* `y_true` — истинные значения целевой переменной.

* `y` — "зашумленные" значения целевой переменной.

На сгенерированной выборке обучите 10 деревьев решений DecisionTreeRegressor со значениями параметра **max_depth от 1 до 10**.

Для каждого обученного дерева рассчитайте MSE на `y` и `y_true`, постройте график предсказаний.

Дополните класс CustomGradientBoostingRegressor, добавив недостающий код там, где это необходимо (**в качестве начального значения используйте среднее**).

Обучите модель `gb_reg_def` (CustomGradientBoostingRegressor) **c параметрами по умолчанию**: n_estimators=100, learning_rate=0.1, max_depth=1.

Выполните подбор оптимальных гиперпараметров для кастомного класса CustomGradientBoostingRegressor с помощью GridSearchCV (см. замечание ниже).

Обучите модель `gb_reg` (CustomGradientBoostingRegressor) с оптимальными гиперпараметрами на полной выборке.

Сравните результаты обучения деревьев с разной глубиной (DecisionTreeRegressor) и моделей `gb_reg_def` и `gb_reg`, рассчитав MSE на `y` и `y_true`, а также построив графики предсказаний для каждой из моделей.

*Класс CustomGradientBoostingRegressor **наследует от класса BaseEstimator**. Класс BaseEstimator в sklearn является базовым классом, предоставляющим встроенные реализации ключевых методов для оптимизации, кросс-валидации и автоматического конфигурирования моделей в рамках библиотеки sklearn. Это позволяет, например, выполнять оптимизацию гипермараметров моделей кастомного (пользовательского) класса CustomGradientBoostingRegressor с помощью GridSearchCV, если в кастомном классе присутствует реализация методов fit, predict и scoring.*

In [ ]:
# Сгенерируйте простой набор данных для регрессии

rng = np.random.RandomState(RANDOM_STATE)
noise = rng.normal(0, 0.18, 1000)
X = pd.DataFrame({'x': np.linspace(1, 10, 1000)})
y_true = -0.3 * X['x'] ** 0.5 * np.cos(np.pi * X['x'] / 4)
y_true = np.array(y_true)
y = y_true + noise

In [ ]:
# Постройте график y и y_true

plt.figure(figsize=(10,6))
plt.scatter(X['x'], y, marker='.', c='#97BFB4', label='y')
plt.plot(X['x'], y_true, c='#DD4A48', label='y true')
plt.legend()
plt.show()

In [ ]:
# Обучите 10 деревьев DecisionTreeRegressor со значениями параметра max_depth от 1 до 10
# Для каждого дерева рассчитайте MSE на y и y_true, постройте график предсказаний
# Не забудьте зафиксировать RANDOM_STATE

for i in range(1, 11):
    tree_reg_i = ...
    y_pred_tree_reg_i = ...
    mse = mean_squared_error(...)
    mse_true = mean_squared_error(...)
    plt.figure(figsize=(10,6))
    plt.title(f'Depth: {i}')
    plt.scatter(X['x'], y, marker='.', c='#97BFB4', label='y')
    plt.plot(X['x'], y_true, c='#DD4A48', label='y true')
    plt.plot(X['x'], y_pred_tree_reg_i, c='#4F091D',
             label=f'tree({i}) prediction (MSE={...:.4f}, true MSE={...:.4f})')
    plt.legend()
    plt.show()

In [ ]:
# Дополните класс CustomGradientBoostingRegressor
# Код метода score изменять не нужно
# Не забудьте фиксировать random_state, где это возможно

class CustomGradientBoostingRegressor(BaseEstimator):
    """
    Простой регрессор на основе градиентного бустинга над деревьями решений.

    Аргументы:
        n_estimators (int): Количество деревьев (итераций бустинга). По умолчанию — 100.
        learning_rate (float) Темп обучения (шаг градиентного спуска). По умолчанию — 0.1.
        max_depth (int): Максимальная глубина дерева бустинга. По умолчанию — 1.
        random_state : (int|None): Сид для фиксирования случайного состояния. По умолчанию — None (не фиксировать).

    Атрибуты:
        f0 (float): Начальное предсказание (среднее значение целевой переменной).
        models (list[DecisionTreeRegressor]): Последовательность деревьев, составляющих модель градиентного бустинга.
    """
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=1, random_state=None):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        self.f0 = None
        self.models = []

    def fit(self, X, y):
        """
        Обучает модель градиентного бустинга.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.
            y (numpy.ndarray): Массив значений целевой переменной.

        Возвращает:
            CustomGradientBoostingRegressor: Обученная модель градиентного бустинга.
        """
        self.f0 = ...
        y_pred = np.full_like(y, self.f0)
        for _ in range(self.n_estimators):
            residuals = ...
            tree = DecisionTreeRegressor(...)
            y_pred += ...
            self.models.append(tree)
        return self

    def predict(self, X):
        """
        Предсказывает значения целевой переменной.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.

        Возвращает:
            numpy.ndarray: Массив предсказанных значений целевой переменной.
        """
        y_pred = np.full(X.shape[0], self.f0)
        for model in self.models:
            y_pred += ...
        return y_pred

    def score(self, X, y):
        """
        Вычисляет отрицательное значение MSE (Negative MSE) для прогноза.
        Метод необходим для применения GridSearchCV.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.
            y (numpy.ndarray): Массив значений целевой переменной.

        Возвращает:
            float: Значение Negative MSE.
        """
        return -mean_squared_error(y, self.predict(X))

In [ ]:
# Обучите модель gb_reg_def (CustomBoostRegressor c параметрами по умолчанию)
# Не забудьте зафиксировать RANDOM_STATE

gb_reg_def = ...

In [ ]:
# Для модели gb_reg_def посчитайте MSE на y и y_true, постройте график предсказаний

y_pred_gb_reg_def = ...
mse_gb_reg_def = ...
mse_true_gb_reg_def = ...

plt.figure(figsize=(10,6))
plt.scatter(X['x'], y, marker='.', c='#97BFB4', label='y')
plt.plot(X['x'], y_true, c='#DD4A48', label='y true')
plt.plot(X['x'], y_pred_gb_reg_def, c='#4F091D',
         label=f'CustomGradientBoostingRegressor prediction (MSE={...:.4f}, true MSE={...:.4f})')
plt.legend()
plt.show()

In [ ]:
# Подберите оптимальные гиперпараметры для кастомного класса CustomGradientBoostingRegressor с помощью GridSearchCV
# Не забудьте зафиксировать RANDOM_STATE

params = {
    'n_estimators': [100, 150, 200, 250],
    'learning_rate': [0.1, 0.2, 0.3, 0.4],
    'max_depth': [1, 2, 3]
}
cv = 5

cv_gb_reg = ...

In [ ]:
# Выведите оптимальные гиперпараметры по результатам оптимизации

...

In [ ]:
# Обучите модель gb_reg (CustomBoostRegressor) с оптимальными гиперпараметрами
# Не забудьте зафиксировать RANDOM_STATE

gb_reg = ...

In [ ]:
# Для модели gb_reg посчитайте MSE на y и y_true, постройте график предсказаний

y_pred_gb_reg = ...
mse_gb_reg = ...
mse_true_gb_reg= ...

plt.figure(figsize=(10,6))
plt.scatter(X['x'], y, marker='.', c='#97BFB4', label='y')
plt.plot(X['x'], y_true, c='#DD4A48', label='y true')
plt.plot(X['x'], y_pred_gb_reg, c='#4F091D',
         label=f'CustomGradientBoostingRegressor prediction (MSE={...:.4f}, true MSE={...:.4f})')
plt.legend()
plt.show()

### **XGBoost, LightGBM и CatBoost**

Среди существующих реализаций градиентного бустинга выделяются три наиболее эффективных и популярных алгоритма: XGBoost, LightGBM и CatBoost.

* Extreme Gradient Boosting ([XGBoost](https://xgboost.readthedocs.io/en/latest/), DMLC 2014 г.) — эффективный и гибкий алгоритм градиентного бустинга с поддержкой регуляризации и параллельного обучения, обеспечивающий высокую производительность и универсальность. Алгоритм требует предварительной обработки категориальных признаков (например, One-Hot кодирование).

* Light Gradient-Boosting Machine ([LightGBM](https://lightgbm.readthedocs.io/en/latest/), Microsoft 2016 г.) — алгоритм, который выделяется высокой скоростью обучения и масштабируемостью для больших объёмов данных и многомерных признаков. Алгоритм требует предварительной обработки категориальных признаков.


* Category Boosting ([CatBoost](https://catboost.ai/), Яндекс 2017 г.) — алгоритм, который специализируется на эффективной работе с категориальными признаками и обладает устойчивостью к переобучению благодаря симметричной структуре деревьев. Алгоритм не требует предварительной обработки категориальных признаков — реализована автоматическая обработка категориальных признаков.

Подробнее про эти алгоритмы можно почитать по **ссылкам**:

* [XGBoost, LightGBM или CatBoost - какой алгоритм бустинга следует использовать? | vk.com](https://vk.com/@coeusds-xgboost-lightgbm-ili-catboost-kakoi-algoritm-bustinga-sled)

### **Датасет *Employee dataset***

**Для решения задания 2 рассмотрим датасет [Employee dataset](https://www.kaggle.com/datasets/tawfikelmetwally/employee-dataset).**

Набор данных предназначен для анализа факторов, влияющих на увольнение сотрудников.

Целевая переменная — LeaveOrNot:

* 1 — сотрудник уволился.

* 0 — сотрудник продолжает работать.

Датасет включает в себя признаки:

* Education — уровень образования.

* JoiningYear — год принятия на работу.

* City — город, в котором расположен офис сотрудника.

* PaymentTier — уровень заработной платы (категория).

* Age — возраст.

* Gender — пол.

* EverBenched — был ли сотрудник когда-либо "на скамейке запасных" (не задействован в проектах компании).

* ExperienceInCurrentDomain — опыт работы в текущей должности (в годах).

### ***Задание 2***

Выполните предобработку датасета (см. код) и убедитесь, что по результатам предобработки:

* Датасет разделён на обучающую и тестовую выборки со стратификацией по целевой переменной в соотношении: **train — 75%, test — 25%**.

* Имеются два набора датасетов с признаками:

    * `X_empl_train`, `X_empl_test` — датасеты **без применения One-Hot кодирования**. Используйте для обучения и оценки **модели `catb_empl` (CatBoostClassifier)**.

    * `X_empl_train_onehot`, `X_empl_test_onehot` — датасеты **с применением One-Hot кодирования**. Используйте для обучения и оценки **всех моделей, кроме `catb_empl`**.

С помощью **RandomizedSearchCV** (n_iter=50) выполните поиск оптимальных гиперпараметров (для каждой из модели в одельности), и на оптимальном наборе параметров обучите модели:

* `rf_empl` — случайный лес (RandomForestClassifier). Подбор параметров и обучение на `X_empl_train_onehot`.

* `gb_empl` — градиентный бустинг sklearn (GradientBoostingClassifier). Подбор параметров и обучение на `X_empl_train_onehot`.

* `xgb_empl` — модель XGBoost (XGBClassifier). Подбор параметров и обучение на `X_empl_train_onehot`.

* `lgbm_empl` — модель LightGBM (LGBMClassifier). Подбор параметров и обучение на `X_empl_train_onehot`.

* `catb_empl` — модель CatBoost (CatBoostClassifier). **Подбор параметров и обучение на `X_empl_train`**.

Сравните качество прогноза моделей, рассчитав метрики AUC и f1 на тестовых выборках (`X_empl_test` и `X_empl_test_onehot`).

*На практике количество итераций при оптимизации гиперпараметров с помощью RandomizedSearchCV зависит от сложности модели и размера датасета. Для реальных задач 50 итераций, вероятнее всего, будет недостаточно. В рамках данного задания небольшое число итераций используется для экономии времени.*

In [ ]:
# Считайте набор данных

df_empl = pd.read_csv('employee.csv')
df_empl

In [ ]:
# Информация о типах столбцов в датасете

df_empl.info()


In [ ]:
# Количество уникальных значений в каждом из столбцов датасета

df_empl.nunique()

In [ ]:
# Создайте список категориальных переменных (не включая целевую переменную)

empl_cat_feat = ['Education', 'City', 'PaymentTier', 'Gender', 'EverBenched']

In [ ]:
# Выделите объясняемый фактор в отдельную переменную

X_empl, y_empl = df_empl.drop(columns=['LeaveOrNot']), df_empl['LeaveOrNot']

In [ ]:
# Разделите датасет на обучающую (75%) и тестовую (25%) выборки со стратификацией по целевой переменной
# Напоминание: не забудьте про RANDOM_STATE

X_empl_train, X_empl_test, y_empl_train, y_empl_test = ... # 75/25

In [ ]:
# Закодируйте категориальные признаки числами 0 и 1 с помощью OneHotEncoder
# Выделите отдельные датасеты с закодированными признаками
#   train -> fit_transform
#   test -> transform

empl_encoder = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')

X_empl_train_onehot = ...
X_empl_test_onehot = ...

In [ ]:
# Сформируем таблицу для сравнения качества прогноза моделей на тестовой выборке

empl_results = pd.DataFrame({
    'model': ['rf_empl', 'gb_empl', 'xgb_empl', 'lgbm_empl', 'catb_empl'],
    'auc_roc': np.zeros(5),
    'f1': np.zeros(5)
})
empl_results = empl_results.set_index('model', drop=True)

##### `rf_empl` (RandomForestClassifier)

In [ ]:
# Подберите оптимальные гиперпараметры обучения rf_empl с помощью RandomizedSearchCV
# Не забудьте зафиксировать RANDOM_STATE

estimator = ...
params = {
    'max_depth': range(1, 11),
    'n_estimators': range(50, 300)
}
n_iter = 50
scoring = 'roc_auc'
cv = 5

rf_empl = RandomizedSearchCV(
    ...
    random_state=...,
    n_jobs=-1, # Может ускорить вычисления за счёт параллелизма, не влияет на результат
    refit=True # Переобучает модель на всей выборке после подбора гиперпараметров (по умолчанию True, можно не указывать)
)

In [ ]:
# Результат подбора гиперпараметров для rf_empl

print(f'Итерация: {...}')
print(f'AUC: {...:.4f}')
print(f'Параметры: {...}')

In [ ]:
# Средняя продолжительность обучения rf_empl (сек.)

print('{:.4f}'.format(rf_empl.cv_results_['mean_fit_time'].mean()))

In [ ]:
# На тестовой выборке оцените AUC и f1 для модели rf_empl
# Добавьте результат в empl_results

empl_results.loc['rf_empl', 'auc_roc'] = ...
empl_results.loc['rf_empl', 'f1'] = ...

##### `gb_empl` (GradientBoostingClassifier)

In [ ]:
# Подберите оптимальные гиперпараметры обучения gb_empl с помощью RandomizedSearchCV
# Не забудьте зафиксировать RANDOM_STATE

estimator = ...
params = {
    'max_depth': range(1, 11),
    'n_estimators': range(50, 300),
    'learning_rate': np.linspace(0.05, 0.95, 100)
}
n_iter = 50
scoring = 'roc_auc'
cv = 5

gb_empl = ...

In [ ]:
# Результат подбора гиперпараметров для gb_empl

...

In [ ]:
# Средняя продолжительность обучения gb_empl (сек.)

print('{:.4f}'.format(gb_empl.cv_results_['mean_fit_time'].mean()))

In [ ]:
# На тестовой выборке оцените AUC и f1 для модели gb_empl
# Добавьте результат в empl_results

...

##### `xgb_empl` (XGBClassifier)

In [ ]:
# Подберите оптимальные гиперпараметры обучения xgb_empl с помощью RandomizedSearchCV
# Не забудьте зафиксировать RANDOM_STATE

estimator = ...
params = {
    'max_depth': range(1, 11),
    'n_estimators': range(50, 300),
    'learning_rate': np.linspace(0.05, 0.95, 100)
}
n_iter = 50
scoring = 'roc_auc'
cv = 5

xgb_empl = ...

In [ ]:
# Результат подбора гиперпараметров для xgb_empl

...

In [ ]:
# Средняя продолжительность обучения xgb_empl (сек.)

print('{:.4f}'.format(xgb_empl.cv_results_['mean_fit_time'].mean()))

In [ ]:
# На тестовой выборке оцените AUC и f1 для модели xgb_empl
# Добавьте результат в empl_results

...

##### `lgbm_empl` (LGBMClassifier)

In [ ]:
# Подберите оптимальные гиперпараметры обучения lgbm_empl с помощью RandomizedSearchCV
# Не забудьте зафиксировать RANDOM_STATE

estimator = ... # verbose=-1
params = {
    'max_depth': range(1, 11),
    'n_estimators': range(50, 300),
    'learning_rate': np.linspace(0.05, 0.95, 100)
}
n_iter = 50
scoring = 'roc_auc'
cv = 5

lgbm_empl = ...

In [ ]:
# Результат подбора гиперпараметров для lgbm_empl

...

In [ ]:
# Средняя продолжительность обучения lgbm_empl (сек.)

print('{:.4f}'.format(lgbm_empl.cv_results_['mean_fit_time'].mean()))

In [ ]:
# На тестовой выборке оцените AUC и f1 для модели lgbm_empl
# Добавьте результат в empl_results

...

##### `catb_empl` (CatBoostClassifier)

In [ ]:
# Подберите оптимальные гиперпараметры обучения catb_empl с помощью RandomizedSearchCV
# Не забудьте зафиксировать RANDOM_STATE

estimator = CatBoostClassifier(random_state=RANDOM_STATE, verbose=False, cat_features=...)
params = {
    'max_depth': range(1, 11),
    'n_estimators': range(50, 300),
    'learning_rate': np.linspace(0.05, 0.95, 100)
}
n_iter = 50
scoring = 'roc_auc'
cv = 5

...

In [ ]:
# Результат подбора гиперпараметров для catb_empl

...

In [ ]:
# Средняя продолжительность обучения catb_empl (сек.)

print('{:.4f}'.format(catb_empl.cv_results_['mean_fit_time'].mean()))

In [ ]:
# На тестовой выборке оцените AUC и f1 для модели catb_empl
# Добавьте результат в empl_results

...

##### Сравнение моделей

In [ ]:
# Сравните качество прогноза моделей на тестовых выборках

empl_results

In [ ]:
# Визуализируйте качество прогноза моделей, построив график f1 ~ AUC

plt.figure(figsize=(7, 7))
plt.title('Точность прогноза моделей на тестовой выборке')
ax = sns.scatterplot(data=empl_results, x=..., y=..., hue='model', s=150)
plt.legend()
plt.show()

### **Early Stopping**

Ранняя остановка (Early Stopping) — это универсальный и широко распространённый метод регуляризации, который позволяет эффективно предотвращать переобучение моделей. Суть метода заключается в остановке обучения модели до завершения всех запланированных итераций в случае, если прогнозные способности модели на валидационной выборке перестают улучшаться или начинают ухудшаться.

**Основные принципы:**

* Исходная выборка делится на подвыборки:

    * Обучающая выборка (train).

    * Валидационная выборка (validation) — на ней оценивается качество модели **во время обучения**.

    * Тестовая выборка (test) — используется для финальной оценки модели (не участвует в процессе обучения).

* Мониторинг метрики. Если в процессе обучения метрика не улучшается в течение заданного числа итераций, обучение останавливается (прекращается).

* Сохранение лучшей модели. Во время обучения сохраняются веса модели на той итерации, когда валидационная метрика была наилучшей.

### **Вероятностные методы оптимизации гиперпараметров**

Вероятностные методы оптимизации гиперпараметров — это итерационные методы, которые позволяют находить оптимальные гиперпараметры обучения моделей быстрее и точнее, чем Grid Search и Randomized Search, за счет использования вероятностной модели целевой функции.

На каждой итерации метод оценивает, в какой следующей точке с наибольшей вероятностью будет достигнуто улучшение текущей оценки оптимума. Вероятностные методы особенно полезны при работе со сложными многомерными пространствами гиперпараметров.

**Преимущества вероятностных методов перед Grid Search и Randomized Search:**

* Каждая итерация использует информацию, полученную на предыдущих итерациях.

* Вероятностные методы способны моделировать внутренние зависимости между гиперпараметрами.

* Вероятностные методы позволяют достичь более высокого качества, если было выполнено достаточное количество итераций.

* Гибкость. Вероятностные методы способны работать с непрерывными, дискретными и категориальными параметрами.

**Основным недостатком** вероятностных методов является высокая вычислительная сложность по сравнению с Grid Search и Randomized Search.

Одним из основных вероятностных методов является TPE (Tree-structured Parzen Estimator). TPE реализован в двух наиболее популярных библиотеках для оптимизации гиперпараметров: Optuna и Hyperopt.

Подробнее можно изучить по **ссылкам:**

* [Подбор гиперпараметров | education.yandex.ru](https://education.yandex.ru/handbook/ml/article/podbor-giperparametrov).

* [Optuna vs Hyperopt: Which Hyperparameter Optimization Library Should You Choose? | eptun.ai](https://neptune.ai/blog/optuna-vs-hyperopt).

### **Датасет *Predict Students' Dropout and Academic Success***

**Для решения задания 3 рассмотрим датасет [Predict Students' Dropout and Academic Success](https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success).**

**ВНИМАНИЕ:** При решении задания **используйте файл students.csv** из приложения к ноутбуку, поскольку исходный датасет был изменен авторами курса.

Датасет создан на основе информации из Португальского высшего учебного заведения и содержит информацию о студентах, зачисленных на различные программы бакалавриата: агрономия, дизайн, педагогика и др. Цель датасета — на ранних этапах обучения выявить студентов, находящихся в зоне риска для последующего оказания поддержки. Набор данных включает информацию, известную на момент зачисления студентов (академическая история, демографические и социально-экономические факторы), а также их академическую успеваемость по итогам первого и второго семестров.

Датасет предназначен для решения задачи **многоклассовой классификации**.

Целевая переменная — Target:

* Dropout — студент отчислен.

* Enrolled — студент продолжает обучение.

* Graduate — студент успешно завершил обучение.

Датасет содержит большое количество категориальных признаков, а в целевой переменной присутствует дисбаланс классов.

### ***Задание 3***

Выполните предобработку датасета (см. код).

Обучите модель `catb_stud_def` (CatBoostClassifier) c параметрами `catb_stud_def_params` (для обучения используйте `X_stud_sample_train`).

Обучите модель `catb_stud_es` (CatBoostClassifier) с использованием early stopping ([early_stopping_rounds](https://catboost.ai/docs/en/references/training-parameters/overfitting-detection#early_stopping_rounds) и параметрами `catb_stud_es_params`. Критерий остановки — 50 итераций без увеличения accuracy на валидационной выборке `X_stud_sample_val` (для обучения используйте `X_stud_sample_train`).

Выполните оптимизацию гиперпараметров обучения CatBoostClassifier с помощью optuna. Для этого необходимо определить целевую функцию optuna (objective) **с использованием стратифицированной кросс-валидации ([StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html)) и ранней остановки на каждом фолде кросс-валидации** (критерий остановки не фиксируется на одном значении и является оптимизируемым гиперпараметром). Метрика оптимизации гиперпараметров — среднее значение **accuracy** на валидации по результатам кросс-валидации.

На оптимальном наборе признаков и всей обучающей выборке обучите модель `catb_stud` (CatBoostClassifier).

На тестовой выборке постройте отчёт по метрикам классификации для моделей `catb_stud_def`, `catb_stud_es` и `catb_stud`.

*На практике количество итераций при оптимизации гиперпараметров с помощью optuna или hyperopt зависит от сложности модели и размера датасета. **Для реальных задач 50 итераций, вероятнее всего, будет недостаточно**. В рамках данного задания небольшое число итераций используется для экономии времени.*

In [ ]:
# Считайте набор данных

df_stud = pd.read_csv('students.csv')
df_stud

In [ ]:
# В датасете присутствует дисбаланс классов

df_stud['Target'].value_counts(normalize=True)

In [ ]:
# Все категориальные признаки описаны числами

df_stud.info()

In [ ]:
# Количество уникальных значений в каждом из столбцов датасета

df_stud.nunique()

In [ ]:
# Создайте список категориальных переменных (не включая целевую переменную)

stud_cat_feat = ['Marital status', 'Application mode', 'Course', 'Daytime/evening attendance', 'Previous qualification',
                   'Nationality', "Mother's qualification", "Father's qualification", 'Displaced', 'Educational special needs',
                   'Debtor', 'Tuition fees up to date', 'Gender', 'Scholarship holder', 'International']
stud_cat_feat

In [ ]:
# Выделите объясняемый фактор в отдельную переменную

X_stud, y_stud = ...

In [ ]:
# Разделите датасет на обучающую (50%) и тестовую (50%) выборки со стратификацией по целевой переменной
# Не забудьте зафиксировать RANDOM_STATE

X_stud_train, X_stud_test, y_stud_train, y_stud_test = ... # 50/50

In [ ]:
# Разделите обучающую выборку (train) на две подвыборки со стратификацией по целевой переменной (по y_stud_train):
#   1. sample_train (75% от train) — для обучения модели
#   2. sample_val (25% от train) — для валидации модели в процессе обучения
# Не забудьте зафиксировать RANDOM_STATE

X_stud_sample_train, X_stud_sample_val, y_stud_sample_train, y_stud_sample_val = ... # 75/25

In [ ]:
# Обучите модель catb_stud_def
# Для обучения используйте sample_train, sample_val также укажите для визуализации обучения

catb_stud_def_params = {
    'cat_features': stud_cat_feat,
    'eval_metric': 'Accuracy', # Метрика для оценки качества на валидации (при early stopping)
    'random_state': RANDOM_STATE,
    'verbose': False
}

catb_stud_def = CatBoostClassifier(...).fit(..., eval_set=(...))

In [ ]:
# Визуализируйте изменение ошибки и accuracy в процессе обучения catb_stud_def

catb_stud_def_evals_result = catb_stud_def.get_evals_result()

fig, ax = plt.subplots(2, 1, figsize=(12, 10))

ax[0].plot(catb_stud_def_evals_result['learn']['MultiClass'], label='train', color='blue')
ax[0].plot(catb_stud_def_evals_result['validation']['MultiClass'], label='val', color='green')
ax[0].set_xlabel('Iterations')
ax[0].set_ylabel('MultiClass')
ax[0].legend()
ax[0].grid()

ax[1].plot(catb_stud_def_evals_result['learn']['Accuracy'], label='train', color='blue')
ax[1].plot(catb_stud_def_evals_result['validation']['Accuracy'], label='val', color='green')
ax[1].set_xlabel('Iterations')
ax[1].set_ylabel('Accuracy')
ax[1].legend()
ax[1].grid()

plt.legend()
plt.show()

In [ ]:
# Постройте отчёт по метрикам классификации для модели catb_stud_def на тестовой выборке

print(classification_report(..., digits=4))

In [ ]:
# Обучите модель catb_stud_es c ранней остановкой после 50 итераций
# Для обучения и валидации используйте sample_train и sample_val

catb_stud_es_params = {
    'cat_features': ...,
    'eval_metric': 'Accuracy',
    'random_state': RANDOM_STATE,
    'verbose': False
}

catb_stud_es = CatBoostClassifier(...).fit(..., eval_set=(...), early_stopping_rounds=...)

In [ ]:
# Визуализируйте изменение ошибки и accuracy в процессе обучения catb_stud_es

catb_stud_es_evals_result = ...

fig, ax = plt.subplots(2, 1, figsize=(12, 10))

ax[0].plot(catb_stud_es_evals_result['learn']['MultiClass'], label='train', color='blue')
ax[0].plot(catb_stud_es_evals_result['validation']['MultiClass'], label='val', color='green')
ax[0].set_xlabel('Iterations')
ax[0].set_ylabel('MultiClass')
ax[0].legend()
ax[0].grid()

ax[1].plot(catb_stud_es_evals_result['learn']['Accuracy'], label='train', color='blue')
ax[1].plot(catb_stud_es_evals_result['validation']['Accuracy'], label='val', color='green')
ax[1].set_xlabel('Iterations')
ax[1].set_ylabel('Accuracy')
ax[1].legend()
ax[1].grid()

plt.show()

In [ ]:
# Постройте отчёт по метрикам классификации для модели catb_stud_es на тестовой выборке

...

In [ ]:
# Определите целевую функцию objective для оптимизации параметров с помощью optuna
# Не забудьте фиксировать random_state, где это возможно

def objective(trial, X, y, cat_features, cv=4, random_state=None):
    """
    Целевая функция для оптимизации гиперпараметров CatBoostClassifier с помощью optuna.

    Аргументы:
        trial (optuna.trial.Trial): Объект trial для предложения гиперпараметров.
        X (pandas.DataFrame): Таблица с признаками.
        y (array-like): Массив значений целевой переменной.
        cat_features (list[str]): Список с категориальными признаками.
        cv (int): Количество фолдов для стратифицированной кросс-валидации. По умолчанию — 5.
        random_state : (int|None): Сид для фиксирования случайного состояния. По умолчанию — None (не фиксировать).
    """
    params = {
    'learning_rate': trial.suggest_float('learning_rate', 0.05, 1, log=True), # Темп обучения: float между 0.05 и 1 в логарифмическом масштабе
    'max_depth': trial.suggest_int('max_depth', 1, 8), # Максимальная глубина деревьев: int между 1 и 8
    'n_estimators': ..., # Количество деревьев в ансамбле: int между 100 и 800
    'colsample_bylevel': ..., # Доля признаков, используемых для построения каждого уровня дерева: float между 0.05 и 1.0

    'cat_features': ...,
    'eval_metric': 'Accuracy',
    'random_state': random_state,
    'verbose': False
    }

    early_stopping_rounds = trial.suggest_int(...) # Критерий остановки: int между 20 и 100

    accuracy_scores = []

    skf = StratifiedKFold(n_splits=..., shuffle=True, random_state=...)

    for train_idx, val_idx in skf.split(...):
        X_train, X_val = ...
        y_train, y_val = ...
        model = CatBoostClassifier(...).fit(..., eval_set=(...), ...)
        accuracy_scores.append(...) # val

    return np.mean(accuracy_scores)

In [ ]:
# Оптимизируйте гиперпараметры модели с помощью optuna (sampler — TPESampler)
# Для подбора гиперпараметров используйте train
# Не забудьте зафиксировать RANDOM_STATE (в seed TPESampler)

n_trials = 60 # Количество тестируемых комбинаций параметров
cv = 4 # Количество фолдов при кросс-валидации

stud_sampler = TPESampler(seed=RANDOM_STATE)

stud_study = optuna.create_study(
    direction='maximize', # Максимизация accuracy
    sampler=stud_sampler,
    study_name='CatBoostClassifier'
)

stud_study.optimize(lambda trial: objective(trial, X_stud_train, y_stud_train, cat_features=stud_cat_feat, cv=..., random_state=...),
    n_trials=...)

In [ ]:
# Визуализируйте важность гиперпараметров обучения после оптимизации

param_importance = get_param_importances(stud_study)

plt.bar(..., ...)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Выведете оптимальные гиперпараметры

stud_study.best_params

In [ ]:
# Обучите модель catb_stud c оптимальными гиперпараметрами
# Для обучения и валидации используйте sample_train и sample_val

catb_stud_params = {
    'cat_features': stud_cat_feat,
    'eval_metric': 'Accuracy',
    'random_state': RANDOM_STATE,
    'verbose': False
}

catb_stud = ...

In [ ]:
# Постройте отчёт по метрикам классификации для модели catb_stud на тестовой выборке

...

In [ ]:
# Определите наиболее влиятельный признак в модели catb_stud

stud_importance = pd.DataFrame({
    'feat_name': catb_stud.feature_names_,
    'feat_importance': catb_stud.get_feature_importance(),
})

stud_importance.sort_values(by=['feat_importance'], ascending=False)